In [ ]:
import os

from pydrake.geometry import StartMeshcat
from pydrake.visualization import ModelVisualizer
from manipulation.station import MakeHardwareStation, load_scenario
from pydrake.all import (
    Simulator,
)

In [ ]:
meshcat = StartMeshcat()

In [ ]:
your_model_filename = "/Users/udayan/Documents/SEAS/6.4212/Psets/7Pset/Tree/tree.sdf" # Write the absolute path to your file here
if your_model_filename:
    visualizer = ModelVisualizer(meshcat=meshcat)
    visualizer.parser().AddModels(your_model_filename)
    visualizer.Run(loop_once=True)

In [ ]:
scenario_data = f"""
directives:
- add_model:
    name: your_model
    file: file://{your_model_filename}
    default_free_body_pose:
        branch_a: # Change here!
            translation: [0, 0, 1]
            rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_frame:
    name: tree_on_world
    X_PF:
        base_frame: world
        translation: [0, 0, 0.5]
        rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_weld:
    parent: tree_on_world
    child: your_model::branch_a    
"""
    
scenario = load_scenario(data=scenario_data)
station = MakeHardwareStation(scenario, meshcat)
simulator = Simulator(station)
meshcat.StartRecording()
simulator.AdvanceTo(2.0)
meshcat.PublishRecording()
